In [90]:
import os
from dotenv import load_dotenv
from jupyterlab.semver import replace_tilde

load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI
os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_KEY")
os.environ["HF_API_KEY"] = os.getenv("HF_TOKEN")

In [106]:
from langchain_core.documents import Document

documents = [
    Document(page_content="Dogs are loyal and friendly, making them popular family pets and needs care.", metadata={"source": "mammal-pets-doc"}),
    Document(page_content="Cats are independent and love quiet spaces to nap and needs minimum care.", metadata={"source": "mammal-pets-doc"}),
    Document(page_content="Goldfish are easy to care for and ideal for beginners.", metadata={"source": "fish-pets-doc"}),
    Document(page_content="Parrots can mimic human speech and enjoy social interaction.", metadata={"source": "bird-pets-doc"}),
    Document(page_content="Rabbits need spacious hutches and daily playtime.", metadata={"source": "mammal-pets-doc"}),
    Document(page_content="Hamsters are small, nocturnal pets that enjoy tunnels.", metadata={"source": "mammal-pets-doc"}),
    Document(page_content="Guinea pigs are gentle rodents that thrive in pairs and easy to care.", metadata={"source": "mammal-pets-doc"}),
    Document(page_content="Budgerigars are colorful parrots that can learn words.", metadata={"source": "bird-pets-doc"}),
    Document(page_content="Betta fish are vibrant and prefer living alone.", metadata={"source": "fish-pets-doc"}),
    Document(page_content="Canaries are songbirds known for their cheerful tunes.", metadata={"source": "bird-pets-doc"}),
    Document(page_content="Turtles need a warm tank with both land and water areas and are to care.", metadata={"source": "reptile-pets-doc"}),
    Document(page_content="Ferrets are playful pets that need lots of supervision.", metadata={"source": "mammal-pets-doc"}),
    Document(page_content="Hedgehogs are shy creatures that enjoy exploring at night.", metadata={"source": "mammal-pets-doc"}),
    Document(page_content="Iguanas need a large, warm habitat with UV lighting.", metadata={"source": "reptile-pets-doc"}),
    Document(page_content="Snakes can be calm pets but require secure enclosures.", metadata={"source": "reptile-pets-doc"}),
    Document(page_content="Hermit crabs live in shells and need humid environments.", metadata={"source": "crustacean-pets-doc"}),
    Document(page_content="Chinchillas have soft fur and need dust baths.", metadata={"source": "mammal-pets-doc"}),
    Document(page_content="Cockatiels are affectionate parrots with crested heads.", metadata={"source": "bird-pets-doc"}),
    Document(page_content="Tarantulas are low-maintenance but not cuddly pets.", metadata={"source": "arachnid-pets-doc"}),
    Document(page_content="Geckos are friendly reptiles that enjoy climbing.", metadata={"source": "reptile-pets-doc"}),
    Document(page_content="Axolotls are unique amphibians that live entirely underwater.", metadata={"source": "amphibian-pets-doc"}),
    Document(page_content="Lovebirds are small parrots that bond strongly with owners and are easy to care.", metadata={"source": "bird-pets-doc"}),
    Document(page_content="Sugar gliders are nocturnal gliding marsupials.", metadata={"source": "mammal-pets-doc"}),
    Document(page_content="Finches are tiny birds that live happily in groups.", metadata={"source": "bird-pets-doc"}),
    Document(page_content="Pythons can grow large and need experienced keepers.", metadata={"source": "reptile-pets-doc"}),
    Document(page_content="Koi fish are ornamental pond fish loved for their colors.", metadata={"source": "fish-pets-doc"}),
    Document(page_content="Degu are social rodents that enjoy chewing and climbing.", metadata={"source": "mammal-pets-doc"}),
    Document(page_content="Box turtles can live decades with proper care.", metadata={"source": "reptile-pets-doc"}),
    Document(page_content="Macaws are large parrots with bright plumage.", metadata={"source": "bird-pets-doc"}),
    Document(page_content="Salamanders thrive in moist habitats with hiding spots.", metadata={"source": "amphibian-pets-doc"}),
]


In [98]:
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000022D103EDD00>, default_metadata=(), model_kwargs={})

In [110]:
from langchain_ollama import OllamaEmbeddings
embeddings=OllamaEmbeddings(model="all-MiniLM")
embeddings

OllamaEmbeddings(model='all-MiniLM', base_url=None, client_kwargs={}, async_client_kwargs={}, sync_client_kwargs={}, mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, keep_alive=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None)

In [111]:
from langchain_community.vectorstores import Chroma
db=Chroma.from_documents(documents,embeddings)
db

In [120]:
result=db.similarity_search("easy to care")
result

[Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are easy to care for and ideal for beginners.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are easy to care for and ideal for beginners.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are easy to care for and ideal for beginners.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are easy to care for and ideal for beginners.')]

await is used with async functions (coroutines) that do I/O operations (like database queries, API calls) non-blocking, so your app can do other things while waiting for the response.

asimilarity_search is just the asynchronous version of similarity_search

similarity_search(query: str) → sync

asimilarity_search(query: str) → async

In [124]:
result=await db.asimilarity_search_with_score("easy to care",k=2)
result

[(Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are easy to care for and ideal for beginners.'),
  1.323835849761963),
 (Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are easy to care for and ideal for beginners.'),
  1.323835849761963)]

### Retrievers
LangChain VectorStore objects do not subclass Runnable, and so cannot immediately be integrated into LangChain Expression Language chains.

LangChain Retrievers are Runnables, so they implement a standard set of methods (e.g., synchronous and asynchronous invoke and batch operations) and are designed to be incorporated in LCEL chains.

We can create a simple version of this ourselves, without subclassing Retriever. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the similarity_search method:

Although don't use this method

In [125]:
from typing import List
from langchain_core.runnables import RunnableLambda
retriever=RunnableLambda(db.similarity_search).bind(k=1)
retriever.batch(["easy to care","cat"])

[[Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are easy to care for and ideal for beginners.')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]]

Use this

In [127]:
retriever=db.as_retriever(search_type="similarity",search_kwargs={"k":1})
retriever.batch(["easy to care","cat"])

[[Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are easy to care for and ideal for beginners.')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]]

So when you run .invoke():

Input → "tell me about dogs"

RunnablePassthrough() → passes "tell me about dogs" as {question}

retriever → fetches relevant docs for "tell me about dogs" → {context}

prompt → fills template with {question} and {context}

llm → gets the final prompt → returns the answer.

In [141]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
message ="""
Answer the question from the context provided only
{question}

Context:
{context}
"""
prompt=ChatPromptTemplate.from_messages(["human",message]) #treat as if human has asked
rag_chain={"context":retriever,"question":RunnablePassthrough()}|prompt|llm
response=rag_chain.invoke("tell me something about dogs")
response





AIMessage(content='Dogs are great companions, known for their loyalty and friendliness.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--f1c45824-8a0c-4b6e-a77b-6e6d053c85f2-0', usage_metadata={'input_tokens': 53, 'output_tokens': 13, 'total_tokens': 66, 'input_token_details': {'cache_read': 0}})